In [0]:
from pyspark.sql.functions import current_date, lit, col

spark.conf.set("fs.azure.account.key.formula1dl2025pro.dfs.core.windows.net", "40wTqDrI1SrXYZ4ZmQnlF6I2CcmoMiye21CqzUrpaiFW0Oj1UtePNjGtFHh9hqMa8c1H5fYPo7PF+AStDyxCCQ==")

In [0]:

df_owners = spark.read.csv("abfss://demo@formula1dl2025pro.dfs.core.windows.net/owners.csv",header=True,inferSchema=True)

df_pets = spark.read.json("abfss://demo@formula1dl2025pro.dfs.core.windows.net/pet.json")

df_pets = df_pets.drop("_corrupt_record")
df_pets = df_pets.na.drop()
df_Joined = df_owners.join(df_pets, on="owner_id", how="inner")
df_Joined = df_Joined.withColumnRenamed("age","pet_age")
df_Joined = df_Joined.select("owner_id","owner_name","pet_name","pet_type","pet_age","city")


df_Joined = df_Joined \
    .withColumn("start_date", current_date()) \
    .withColumn("end_date", lit(None).cast("date")) \
    .withColumn("is_current", lit("Y"))

df_Joined.write.format("delta").mode("overwrite").saveAsTable("pets_owners_combined")




In [0]:
df_updated_owners = spark.read.csv("abfss://demo@formula1dl2025pro.dfs.core.windows.net/updated_owners.csv",header=True,inferSchema=True)

df_updated_pets = spark.read.json("abfss://demo@formula1dl2025pro.dfs.core.windows.net/updated_pets.json")

df_updated_pets = df_updated_pets.drop("_corrupt_record") 
df_updated_pets = df_updated_pets.na.drop()
df_source = df_updated_owners.join(df_updated_pets, on="owner_id", how="inner")
df_source = df_source.withColumnRenamed("age","pet_age")
df_source = df_source.select("owner_id","owner_name","pet_name","pet_type","pet_age","city")

df_source = df_source \
    .withColumn("start_date", current_date()) \
    .withColumn("end_date", lit(None).cast("date"))


In [0]:
df_old = spark.table("pets_owners_combined")
df_new = df_source

joined_df = df_new.alias("new").join(
    df_old.alias("old"),
    on="owner_id",
    how="left"
)

columns_to_track = ["owner_id", "owner_name", "pet_name","pet_type", "pet_age", "city"]

from pyspark.sql.functions import sha2, concat_ws, col

joined_df = joined_df.withColumn(
    "hash_new", sha2(concat_ws("||", *[col(f"new.{c}") for c in columns_to_track]), 256)
).withColumn(
    "hash_old", sha2(concat_ws("||", *[col(f"old.{c}") for c in columns_to_track]), 256)
)

changed_new = joined_df.filter(
    (col("hash_old").isNull()) | (col("hash_new") != col("hash_old"))
)

new_records = changed_new.select([col(f"new.{c}").alias(c) for c in columns_to_track])

new_records = new_records.withColumn("start_date", current_date()).withColumn("end_date", lit(None).cast("date")).withColumn("is_current", lit("Y"))

expired_records = df_old.join(
    new_records.select("owner_id").distinct(),
    on="owner_id",
    how="inner"
).filter(col("is_current") == "Y") \
 .withColumn("end_date", current_date()) \
 .withColumn("is_current", lit("N"))


unchanged_records = df_old.join(
    new_records.select(new_records.owner_id),
    on="owner_id",  # clear and safe
    how="left_anti"
).filter("is_current = 'Y'")

final_df = unchanged_records.unionByName(expired_records, allowMissingColumns=True).unionByName(new_records, allowMissingColumns=True)
final_df = final_df.sort("owner_id")

final_df.write.format("delta").mode("overwrite").saveAsTable("pets_owners_combined")





In [0]:
%sql
select * from pets_owners_combined

owner_id,owner_name,pet_name,pet_type,pet_age,city,start_date,end_date,is_current
1,John,Buddy,Dog,3,New York,2022-01-02,null,Y
2,Jane,Mittens,Cat,2,Los Angeles,2022-01-02,2025-04-17,N
2,Jane,Mittens,Cat,3,San Francisco,2025-04-17,null,Y
3,Tom,Max,Dog,4,Chicago,2022-01-02,2025-04-17,N
3,Tom,Max,Dog,5,Austin,2025-04-17,null,Y
4,Mary,Bella,Cat,5,Houston,2022-01-02,null,Y
5,David,Lucy,Dog,1,Phoenix,2022-01-02,null,Y
6,Eva,Rocky,Dog,7,Philadelphia,2022-01-02,null,Y
7,Grace,Lily,Cat,3,San Antonio,2022-01-02,null,Y
8,Henry,Oscar,Fish,1,San Diego,2022-01-02,null,Y
